In [ ]:
# Imports 

import pandas as pd
import numpy as np
from df2gspread import df2gspread as d2g
import inspect as inspect 
import docx
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from utils import *
from spidercharts import *
from param import *
from company_profiles import *
import matplotlib.pyplot as plt
import matplotlib as mpl
from math import pi


# Get the row of the sections_scores_df corresponding to the company in question

s1 = sections_scores_df.loc["Brembo"]
s1.fillna(value=pd.np.nan, inplace=False)
df1 = s1.to_frame().transpose()
df1 = df1[df1.columns[0:10]]
df1 # it works. 

s2 = grouped_scores_df_transposed.loc[sections_scores_df.loc["Brembo", "Company_Sector"]]
s3 = sections_scores_df.loc["Averages"][0:10]

df2 = s2.to_frame().join(s3.to_frame()).transpose()

df3 = df1.append(df2)
my_df = df3.reset_index().rename(index=int, columns={"index": "groups"})

# We need to rename the columns for the viz to be explicit about what every section is. 
my_df

new_cols_dict = {"Section_1":"Dichiarazioni \nPubbliche", 
                 "Section_2":"Modello \nAnticorruzione", 
                 "Section_3":"Codice",
                 "Section_4":"Whistleblowing", 
                 "Section_5":"Lobbying", 
                 "Section_6":"Conflitto \nd'Interesse",
                 "Section_7":"Finanziamenti \nPolitici",
                 "Section_8":"Traspareza \nOrganizzativa",
                 "Section_9":"Formazione", 
                 "Section_10":"Sostenibilità"}

my_df.rename(new_cols_dict, axis='columns', inplace=True)

my_df.at[1,"groups"] = "Medie Settore ({})".format(my_df.at[1,"groups"])
my_df.at[2,"groups"] = "Medie TOT."

for i in my_df.columns[1:]:
    if my_df.at[0, i] == None:
        new = i+"\n(Disattivata)"
        my_df.rename({i : new}, axis='columns', inplace=True)
        my_df.at[0,new] = 0.01

my_df
my_categories=list(my_df)[1:] 
my_categories

In [ ]:
# We need to find the αrad of each angle to create the spider chart. 
# We will apply the following proportion ==> αrad : α° = 2π : 360° ==> αrad = (α° * 2π)/360° 
# I have 10 points on the circle whose arcs subtend angles each distant from one other exactly 36°. 
# αrad of the first arc = (36° * 2π)/360° = (1 * 2π)/10° 
# 36*2*pi/360 == 1/N*2*pi == 1/len(categories)*2*pi  # True

my_categories=list(my_df)[1:] 
my_N =len(my_categories)
my_angles = [(n * 2 * pi) / my_N for n in range(my_N)] 
my_angles += my_angles[:1]
my_angles

In [ ]:
def plot_indicator(my_df, groups=0, label="", color=""):
            values = my_df.loc[groups].drop('groups').values.flatten().tolist()
            values += values[:1]
            my_ax.plot(my_angles, values, linewidth=1, linestyle='solid', label="{}".format(label))
            my_ax.fill(my_angles, values, '{}'.format(color), alpha=0.1)
            return(my_ax)
        
# Initialise the spider plot
my_fig = plt.figure()
my_ax = my_fig.add_subplot(111,  polar=True)

# If you want the first axis to be on top:
my_ax.set_theta_offset(pi / 2)
my_ax.set_theta_direction(-1)

# Draw one axe per variable + add labels that don't cross the circle line
plt.xticks(my_angles[:-1], my_categories, fontsize=11, color="black", horizontalalignment = "center")

# Draw ylabels
my_ax.set_rlabel_position(1)
plt.yticks([0.25,0.50,0.75], ["25%","50%","75%"], color="grey", size=10)
plt.ylim(0,1)

# Ind1
plot_indicator(my_df, groups=0, label="Brembo", color="b")
# Ind2
plot_indicator(my_df, groups=2, label="Media totale", color="r")

# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(0.01, 0.01))
#plt.savefig('company_only.png', dpi = 300)

In [ ]:
store_file(file = my_fig, file_name = "Brembo_Medie_totali", destination_path = sub_folders_ls[1]+ "/" + "Brembo")

In [ ]:
destination_path = sub_folders_ls[1]+ "/" + "Brembo"
file_name = "Brembo_Medie_totali"
print (root + "/" + destination_path + "/" + file_name + ".png")